In [1]:
using NBInclude
@nbinclude("Hofstadter Single Particle in Julia.ipynb")

Hofstadter_SP (generic function with 1 method)

In [2]:
using QuantumOptics

In [3]:
function Hofstadter_SP_Op(Nx, Ny, alpha)
    
    H_SP = Hofstadter_SP(Nx,Ny,alpha, 0)
    
    N = Nx*Ny
    
    global Basis = NLevelBasis(N)
    
    H = SparseOperator(Basis)

    for m in 1:N
        for n in 1:N
            H = H + H_SP[m,n] * transition(Basis, m, n)
        end
    end
    
    return H
end

Hofstadter_SP_Op (generic function with 1 method)

1) Hofstadter SP QoJulia energies are has to be equal to Hofstadter Single Particle energies.

In [4]:
using LinearAlgebra
eigenenergies(dense(Hofstadter_SP_Op(4, 4, 1/4))) == eigvals(Hofstadter_SP(4,4,1/4,0))

true

In [25]:
function Subspace_Op(SP_Op, N_cut)
    
    E0, states0 = eigenstates(dense(SP_Op))
    states = states0[1:N_cut]
    #states = states0

    global b_sub = SubspaceBasis(Basis,states)
    
    global P = projector(b_sub, Basis)
    global Pt = dagger(P)
    
    Sub_Op = P*SP_Op*Pt
    
    return Sub_Op
end

Subspace_Op (generic function with 1 method)

In [42]:
function Sub_Int_Op(SP_Op, N_cut, PN, U)

    Subspace_Op(SP_Op, N_cut)
    
    global StatesBH = bosonstates(b_sub, [PN]) 
    global Basis_MB = ManyBodyBasis(b_sub, StatesBH)

    IT = SparseOperator(Basis_MB)
    
    N = size(SP_Op,1)
    
    for m in 1:N
        
        NM = transition(Basis, m, m)
        NMP = P*NM*Pt
        
        global NM_MB_2 = SparseOperator(Basis_MB)
        for i in 1:length(b_sub)
            for j in 1:length(b_sub)
                NM_MB_2 = NM_MB_2 + NMP.data[i,j] * transition(Basis_MB, i, j)
            end
        end

        IT = IT + U/2 * NM_MB_2 * ( NM_MB_2 - identityoperator(Basis_MB) )
    end
    
    return IT
end

Sub_Int_Op (generic function with 1 method)

In [38]:
function Sub_Space_Hofstadter_Finite_U(SP_Op, N_cut, PN, U)
    
    SO = Subspace_Op(SP_Op, N_cut)
    
    SIO = Sub_Int_Op(SP_Op, N_cut, PN, U)
    
    KT = SparseOperator(Basis_MB)
    
    for m in 1:length(b_sub)
        for n in 1:length(b_sub)
            KT = KT + SO.data[m,n] * transition(Basis_MB, m, n)
        end
    end
    
    return H = KT + SIO
end

Sub_Space_Hofstadter_Finite_U (generic function with 1 method)

2) If U<<Band-Gap, Sub-Space Hofstadter Finite-U energies converges at Hofstadter Finite-U energies.
3) When all SP states projected to the new sub-space, Sub-Space Hofstadter Finite-U energies have to equal to Hofstadter Finite-U energies.

In [28]:
@nbinclude("Hofstadter MB in Julia.ipynb"; regex=r"#.*executeme")

Hofstadter_Non_Int (generic function with 1 method)

In [39]:
# using Plots

Nx = Ny = 4
alpha = 1/4
N_cut = 4
PN = 2
U = 0.1

SP_Op = Hofstadter_SP_Op(Nx, Ny, alpha)
x = Sub_Space_Hofstadter_Finite_U(SP_Op, N_cut, PN, U)
E1 = eigenenergies(dense((x+dagger(x))/2))

E2 = eigenenergies(dense(Hofstadter_Finite_U(Nx, Ny, alpha, PN, U)))

# print(E1-E2[1:length(E1)])

# plot(1:length(E1),E1,seriestype=:scatter,markershape=:star5, markersize=6, label="Sub-Space")
# plot!(1:length(E2),E2,seriestype=:scatter,label="Real-Space",legend=:topleft)
# xlabel!("n");ylabel!("E")

136-element Vector{Float64}:
 -5.65632808701004
 -5.656328087010034
 -5.650720758234421
 -5.650720758234409
 -5.644567583386063
 -5.644567583386059
 -5.644567583386055
 -5.644567583386051
 -5.638898559495862
 -5.638898559495853
 -2.8284271247461925
 -2.8284271247461925
 -2.828427124746191
  ⋮
  2.878869031963123
  2.8788690319631245
  5.657400802749862
  5.6574008027498754
  5.663224889176773
  5.663224889176782
  5.669573714305009
  5.669573714305014
  5.669573714305022
  5.669573714305025
  5.675331653843849
  5.675331653843852

In [40]:
E1

10-element Vector{Float64}:
 -5.731318084257213
 -5.731318084257209
 -5.72560424949238
 -5.725604249492376
 -5.71935424949238
 -5.719354249492379
 -5.719354249492379
 -5.719354249492375
 -5.713640414727549
 -5.7136404147275455